In [1]:
# importing libraries
import xarray as xr
import geopandas as gpd
import numpy as np
import regionmask
from pyproj import CRS


In [2]:
# specifying paths
shapefile_path = "C:\jay projects\school\COMP 4560\CGC_Grain_Outcome_Predictions\src\MoistureSatelliteTest\lpr_000b16a_e.shp"
netcdf_file_path = "C:\jay projects\school\COMP 4560\CGC_Grain_Outcome_Predictions\src\MoistureSatelliteTest\perfectDemo.nc"

In [3]:
# reading shape file
shapefile = gpd.read_file(shapefile_path)

In [29]:
shapefile

,PRUID,PRNAME,PRENAME,PRFNAME,PREABBR,PRFABBR,geometry
0,10,Newfoundland and Labrador / Terre-Neuve-et-Lab...,Newfoundland and Labrador,Terre-Neuve-et-Labrador,N.L.,T.-N.-L.,"MULTIPOLYGON (((8307365.589 2582136.711, 83083..."
1,11,Prince Edward Island / Île-du-Prince-Édouard,Prince Edward Island,Île-du-Prince-Édouard,P.E.I.,Î.-P.-É.,"MULTIPOLYGON (((8435711.754 1679935.966, 84358..."
2,12,Nova Scotia / Nouvelle-Écosse,Nova Scotia,Nouvelle-Écosse,N.S.,N.-É.,"MULTIPOLYGON (((8470851.646 1624745.011, 84710..."
3,13,New Brunswick / Nouveau-Brunswick,New Brunswick,Nouveau-Brunswick,N.B.,N.-B.,"MULTIPOLYGON (((8176224.537 1722553.460, 81762..."
4,24,Quebec / Québec,Quebec,Québec,Que.,Qc,"MULTIPOLYGON (((8399709.494 2261445.703, 84005..."
5,35,Ontario,Ontario,Ontario,Ont.,Ont.,"MULTIPOLYGON (((6378815.614 2295412.440, 63787..."
6,46,Manitoba,Manitoba,Manitoba,Man.,Man.,"MULTIPOLYGON (((6039656.509 2636304.343, 60396..."
7,47,Saskatchewan,Saskatchewan,Saskatchewan,Sask.,Sask.,"POLYGON ((5248633.914 2767057.263, 5249285.640..."
8,48,Alberta,Alberta,Alberta,Alta.,Alb.,"POLYGON ((5228304.177 2767597.891, 5228098.463..."
9,59,British Columbia / Colombie-Britannique,British Columbia,Colombie-Britannique,B.C.,C.-B.,"MULTIPOLYGON (((4018904.414 3410247.271, 40194..."


In [4]:
# Match the netCDF CRS (EPSG:4326)
netcdf_crs = CRS.from_epsg(4326)

In [81]:
# Load the netCDF dataset
dataset = xr.open_dataset(netcdf_file_path)
lon = dataset['lon']
lat = dataset['lat']

C:\Users\jaydo\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\coding\times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(


In [6]:
# Reproject the shapefile to match the netCDF CRS
shapefile_reprojected = shapefile.to_crs(netcdf_crs)

In [71]:
# checking for crs in shape file and netCDF file
shapefile_reprojected.crs
dataset

<xarray.Dataset>
Dimensions:     (time: 1, lat: 720, lon: 1440)
Coordinates:
  * time        (time) datetime64[ns] 2004-06-05
  * lat         (lat) float64 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon         (lon) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
Data variables:
    sm          (time, lat, lon) float32 ...
    flag        (time, lat, lon) float32 ...
    freqbandID  (time, lat, lon) float32 ...
    dnflag      (time, lat, lon) float32 ...
    mode        (time, lat, lon) float32 ...
    sensor      (time, lat, lon) float64 ...
    t0          (time, lat, lon) datetime64[ns] ...
Attributes: (12/44)
    title:                        ESA CCI Surface Soil Moisture COMBINED acti...
    institution:                  TU Wien (AUT); VanderSat B.V. (NL)
    contact:                      cci_sm_contact@eodc.eu
    source:                       WARP 5.5R1.1/AMI-WS/ERS12 Level 2 Soil Mois...
    platform:                     Nimbus 7, DMSP, TRMM, AQUA, Coriolis, GCOM-...
    processing_level:             Quality-controlled, super-collocated (L3S) ...
    ...                           ...
    geospatial_lon_units:         degrees_east
    geospatial_lat_resolution:    0.25 degree
    geospatial_lon_resolution:    0.25 degree
    spatial_resolution:           25km
    time_coverage_start:          20040605T000000Z
    time_coverage_end:            20040605T235959Z

In [7]:
# Create a mask for the shapefile
mask = regionmask.mask_geopandas(shapefile_reprojected, dataset.lon, dataset.lat)

In [8]:
# Apply the mask to the dataset
masked_data = dataset.where(mask)

C:\Users\jaydo\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\coding\times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(


In [9]:
# Convert the masked data to a GeoPandas DataFrame
masked_gdf = masked_data.to_dataframe().reset_index()

In [10]:
masked_gdf

,time,lat,lon,sm,flag,freqbandID,dnflag,mode,sensor,t0
0,2004-06-05,89.875,-179.875,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,2004-06-05,89.875,-179.625,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,2004-06-05,89.875,-179.375,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,2004-06-05,89.875,-179.125,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,2004-06-05,89.875,-178.875,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
1036795,2004-06-05,-89.875,178.875,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1036796,2004-06-05,-89.875,179.125,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1036797,2004-06-05,-89.875,179.375,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1036798,2004-06-05,-89.875,179.625,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [11]:
# Print the GeoPandas DataFrame
print(masked_gdf['sm'].count())

137315


In [12]:
masked_gdf['sm']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
1036795   NaN
1036796   NaN
1036797   NaN
1036798   NaN
1036799   NaN
Name: sm, Length: 1036800, dtype: float32

In [13]:
manitoba = shapefile[shapefile['PRNAME'] == "Manitoba"]

In [14]:
manitoba

,PRUID,PRNAME,PRENAME,PRFNAME,PREABBR,PRFABBR,geometry
6,46,Manitoba,Manitoba,Manitoba,Man.,Man.,"MULTIPOLYGON (((6039656.509 2636304.343, 60396..."


In [15]:
manitoba_reprojected = manitoba.to_crs(netcdf_crs)

In [16]:
manitoba_reprojected

,PRUID,PRNAME,PRENAME,PRFNAME,PREABBR,PRFABBR,geometry
6,46,Manitoba,Manitoba,Manitoba,Man.,Man.,"MULTIPOLYGON (((-94.82341 59.99352, -94.82398 ..."


In [17]:
# Create a mask for the shapefile
mask_manitoba = regionmask.mask_geopandas(manitoba_reprojected, dataset.lon, dataset.lat)

In [18]:
# Apply the mask to the dataset
masked_data_manitoba = dataset.where(mask_manitoba)

C:\Users\jaydo\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\coding\times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(


In [19]:
# Convert the masked data to a GeoPandas DataFrame
masked_gdf_manitoba = masked_data_manitoba.to_dataframe().reset_index()

In [20]:
masked_gdf_manitoba['sm']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
1036795   NaN
1036796   NaN
1036797   NaN
1036798   NaN
1036799   NaN
Name: sm, Length: 1036800, dtype: float32

In [21]:
masked_gdf_manitoba['sm'].count()

137713

In [22]:
alberta = shapefile_reprojected[shapefile_reprojected['PRNAME'] == "Alberta"]

In [23]:
alberta

,PRUID,PRNAME,PRENAME,PRFNAME,PREABBR,PRFABBR,geometry
8,48,Alberta,Alberta,Alberta,Alta.,Alb.,"POLYGON ((-110.00000 59.95257, -110.00000 59.9..."


In [83]:
mask_alberta = regionmask.mask_geopandas(alberta, lon, lat)

In [84]:
mask_alberta

<xarray.DataArray 'mask' (lat: 720, lon: 1440)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * lat      (lat) float64 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon      (lon) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
Attributes:
    standard_name:  region

In [85]:
masked_data_alberta = dataset.where(mask_alberta)

C:\Users\jaydo\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\coding\times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(


In [86]:
masked_gdf_alberta = masked_data_alberta.to_dataframe().reset_index()

In [87]:
masked_gdf_alberta['sm'].count()

137713

In [28]:
dataset.lon[0]

<xarray.DataArray 'lon' ()>
array(-179.875)
Coordinates:
    lon      float64 -179.9
Attributes:
    standard_name:        longitude
    units:                degrees_east
    valid_range:          [-180.  180.]
    _CoordinateAxisType:  Lon

In [78]:
np.array_equal(masked_gdf_manitoba, masked_gdf_alberta)

False

In [80]:
test = masked_gdf_alberta.copy()
test_sm = test[['lon','lat','sm']]
test_sm = test_sm.dropna()
test_sm

,lon,lat,sm
109277,139.375,71.125,0.020541
110259,24.875,70.875,0.239877
110260,25.125,70.875,0.239723
110261,25.375,70.875,0.234124
110262,25.625,70.875,0.241177
...,...,...,...
816918,-70.375,-51.875,0.232667
816919,-70.125,-51.875,0.237066
816920,-69.875,-51.875,0.234583
816921,-69.625,-51.875,0.220140


In [79]:
test2 = masked_gdf_manitoba.copy()
test2_sm = test2['sm']
test2_sm = test2_sm.dropna()
test2_sm

109277    0.020541
110259    0.239877
110260    0.239723
110261    0.234124
110262    0.241177
            ...   
816918    0.232667
816919    0.237066
816920    0.234583
816921    0.220140
816922    0.229484
Name: sm, Length: 137713, dtype: float32

In [61]:
test3 = masked_gdf.copy()
test3_sm = test3['sm']
test3_sm = test3_sm.dropna()
test3_sm

109277    0.020541
110259    0.239877
110260    0.239723
110261    0.234124
110262    0.241177
            ...   
816918    0.232667
816919    0.237066
816920    0.234583
816921    0.220140
816922    0.229484
Name: sm, Length: 137315, dtype: float32

<xarray.DataArray 'lon' (lon: 1440)>
array([-179.875, -179.625, -179.375, ...,  179.375,  179.625,  179.875])
Coordinates:
  * lon      (lon) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
Attributes:
    standard_name:        longitude
    units:                degrees_east
    valid_range:          [-180.  180.]
    _CoordinateAxisType:  Lon